##DDPG 

In [1]:
#https://stackoverflow.com/questions/64161280/rl-problem-on-colab-for-gym-envs-box2d-has-no-attribute-lunarlander
!pip3 install box2d-py
!pip3 install gym[Box_2D]
import gym
env = gym.make("LunarLanderContinuous-v2")

     |████████████████████████████████| 448 kB 18.3 MB/s 


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
import copy 
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [3]:
#https://github.com/openai/baselines/blob/master/baselines/ddpg/noise.py

class OUActionNoise():
  def __init__(self, mu, sigma=0.15, theta=0.2, dt=1e-2, x0=None):
    self.mu = mu
    self.theta = theta
    self.dt = dt
    self.x0 = x0  #starting value
    self.sigma = sigma
    self.reset()  #reset noise at the top of each episode (reset noise) at our constructor
  
  def __call__(self):
    #What __call__ enables us to do:
    #noise = OUActionNoise()  
    #current_noise = noise()

    #GET TEMPORAL correlation of noise
    x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
    self.sigma  * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
  
    self.x_prev = x
    return x

  def reset(self):
    self.x_prev = self.xo if self.x0 is not None else np.zeros_like(self.mu)

    

In [4]:
#THIS CODE along with the rest of the enviro can be used for ANY atari
#game since it is totally generic - copied from DQN course since I 've already 
#dedicated a significant amount of time to write this in the first place
class ReplayBuffer():
  def __init__(self, max_size, input_shape, n_actions):
    self.mem_size = max_size 

    #position of last stored memory
    self.mem_cntr = 0

    #input_shape is for handling states of variable length 
    #DTYPE is for pytorch particulars plus 64 bit too much memory
    self.state_memory = np.zeros((self.mem_size, *input_shape), dtype=np.float32)
    
    self.new_state_memory = np.zeros((self.mem_size, *input_shape), dtype=np.float32)

    #n_Actions here really is the #of componenets per action, here in LunarLander we have a vector of 4 items
    self.action_memory = np.zeros((self.mem_size, n_actions), dtype=np.int64)

    self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)

    #boolean since it will be used as a mask for actor-critic in learn() 
    self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)

  def store_transition(self,state ,action, reward, state_, done):
    
    #cleverer way to handle it, what they did in GDRL
    index = self.mem_cntr % self.mem_size
    self.state_memory[index] = state
    self.action_memory[index] = action
    self.reward_memory[index] = reward
    self.new_state_memory[index] = state_
    self.terminal_memory[index] = done
    self.mem_cntr += 1

  def sample_buffer(self, batch_size):

    #sample only meaningful entries, i.e those that have been filled 
    max_mem = min(self.mem_cntr, self.mem_size)

    batch = np.random.choice(max_mem, batch_size) #, replace=False)

    states = self.state_memory[batch]
    actions = self.action_memory[batch]
    rewards = self.reward_memory[batch]
    next_states = self.new_state_memory[batch]
    dones = self.terminal_memory[batch]

    return states, actions, rewards, next_states, dones

In [6]:
#https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch
#https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch
#https://discuss.pytorch.org/t/batch-normalization-of-linear-layers/20989
#https://androidkt.com/use-the-batchnorm-layer-in-pytorch/

class CriticNetwork(nn.Module):
  #CRITIC network wants to output Q values!
  def __init__(self, input_dims, n_actions, f1=400, f2=300 ,name="None", chkpt_dir="None", lr=0.001):
    super(CriticNetwork, self).__init__()

    self.input_dims = input_dims

    self.checkpoint_dir = chkpt_dir
    self.checkpoint_file = os.path.join(self.checkpoint_dir, name)


    #two hidden layers means the input-400 and the 400-300 ones, right? was confused by that erlier aw well: YES, its correct
    
    self.h1 = nn.Linear(*input_dims, f1)
    torch.nn.init.uniform_(self.h1.weight, -1/np.sqrt(f1), 1/np.sqrt(f1) )
    torch.nn.init.uniform_(self.h1.bias, -1/np.sqrt(f1), 1/np.sqrt(f1) )

    self.bn1 = nn.LayerNorm(f1)


    #ALtenrative way to intitialize weights
    #self.h1.weight.data.uniform_(-3, 3)

    self.h2 = nn.Linear(f1, f2)
    torch.nn.init.uniform_(self.h2.weight, -1/np.sqrt(f2), 1/np.sqrt(f2) )
    torch.nn.init.uniform_(self.h2.bias, -1/np.sqrt(f2), 1/np.sqrt(f2) )   #forgot to add biases 

    self.bn2 = nn.LayerNorm(f2) #need batch norm on second layer as well? isn't that where we concat? wtf
    
    #as i thought more likely (though still not sure) output is only 1 node, the q-value of the input state
    self.output = nn.Linear(f2, 1)
    torch.nn.init.uniform_(self.output.weight, -3e-3, 3e-3 )
    torch.nn.init.uniform_(self.output.bias, -3e-3, 3e-3 )


    self.action_value = nn.Linear(n_actions, f2) #how do i need that?

    self.optimizer = optim.Adam(self.parameters(), lr=lr, weight_decay=0.01 )
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.to(self.device)


  def forward(self, state, action):
    state = np.expand_dims(state, axis=0)
    if not torch.is_tensor(state):
      state = torch.tensor(state).float().to(self.device)  

    if not torch.is_tensor(action):
      action = torch.tensor(action).float().to(self.device)  
    
    out = F.relu(self.bn1(self.h1(state)))
   
   #the part I didn't understand with the action value
    state_value =  self.bn2(self.h2(out))
    
    action_value = self.action_value(action)
    
    #I truly don't understand why that works
    state_action_value  = F.relu(torch.add(action_value, state_value))

    state_action_value = self.output(state_action_value)

    return state_action_value


  def save_checkpoint(self):
    print("...saving checkpoint...")
    torch.save(self.state_dict(), self.checkpoint_file)

  def load_checkpoint(self):
    print("...loading checkpoint")
    self.load_state_dict(torch.load(self.checkpoint_file))


In [7]:
class ActorNetwork(nn.Module):
  def __init__(self, input_dims, n_actions, f1=400, f2=300 ,name="None", chkpt_dir="None", lr=1e-4):
    super(ActorNetwork, self).__init__()

    self.input_dims = input_dims

    self.checkpoint_dir = chkpt_dir
    self.checkpoint_file = os.path.join(self.checkpoint_dir, name)
    
    self.h1 = nn.Linear(*input_dims, f1)
    torch.nn.init.uniform_(self.h1.weight, -1/np.sqrt(f1), 1/np.sqrt(f1) )
    torch.nn.init.uniform_(self.h1.bias, -1/np.sqrt(f1), 1/np.sqrt(f1) )

    self.bn1 = nn.LayerNorm(f1)  #LayerNorm normalizes inputs like batchNorm BUT w/out depending on batch size  
                                   #and also seems to work better with the pytorch framework

    self.h2 = nn.Linear(f1, f2)
    torch.nn.init.uniform_(self.h2.weight, -1/np.sqrt(f2), 1/np.sqrt(f2) )
    torch.nn.init.uniform_(self.h2.bias, -1/np.sqrt(f2), 1/np.sqrt(f2) )   #didnt forget to add biases this time

    self.bn2 = nn.LayerNorm(f2) 
    
    self.output = nn.Linear(f2, n_actions)
    torch.nn.init.uniform_(self.output.weight, -3e-3, 3e-3 )
    torch.nn.init.uniform_(self.output.bias, -3e-3, 3e-3 )



    self.optimizer = optim.Adam(self.parameters(), lr=lr )
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.to(self.device)


  def forward(self, state):
    state = np.expand_dims(state, axis=0)
    if not torch.is_tensor(state):
      state = torch.tensor(state).float().to(self.device)  
    out = F.relu(self.bn1(self.h1(state)))
    out = F.relu(self.bn2(self.h2(out)))
    
    #should multiply this by the action bounds of my env since it is (-1,1) 
    #and env might have higher intervals, eg (-2, 2) (but not necessary in Lunar Lander)
    #leading me to getting only part of environment's action space 
   
    #ALSO apparently F.tanh is deprecated so we will use torch.tanh (though don't know why)
    out = torch.tanh(self.output(out))
    return out


  def save_checkpoint(self):
    print("...saving checkpoint...")
    torch.save(self.state_dict(), self.checkpoint_file)

  def load_checkpoint(self):
    print("...loading checkpoint")
    self.load_state_dict(torch.load(self.checkpoint_file))


In [10]:
class DDPGAgent():
  
  def __init__(self, input_dims, n_actions, lr_actor= 1e-4 , lr_critic= 1e-3 , gamma=0.99, 
              f1_actor=400, f2_actor=300, f1_critic = 400, f2_critic=300,
              name="None", chkpt_dir="/content", max_mem=25000, batch_size=200, tau=0.01 ):
  
    self.gamma  = gamma
    self.n_actions = n_actions
    self.input_dims = input_dims
    self.tau = tau 

    self.lr_actor = lr_actor
    self.lr_critic = lr_critic 

    self.f1_actor = f1_actor
    self.f2_actor = f2_actor 
    

    self.f1_critic = f1_critic
    self.f2_critic = f2_critic 

    self.max_mem = max_mem
    self.batch_size = batch_size
    self.chkpt_dir = chkpt_dir 
    self.name = name

    self.actor = ActorNetwork(self.input_dims, self.n_actions, self.f1_actor, 
                              self.f2_actor, self.name + "_" + "actor", 
                              self.chkpt_dir, lr=self.lr_actor)

    self.actor_target = ActorNetwork(self.input_dims, self.n_actions, self.f1_actor, 
                              self.f2_actor, self.name + "_" + "actor_target", 
                              self.chkpt_dir, lr=self.lr_actor)


    self.critic = CriticNetwork(self.input_dims, self.n_actions, self.f1_critic, 
                              self.f2_critic, self.name + "_"+ "critic", 
                              self.chkpt_dir, lr= self.lr_critic)
    
    self.critic_target = CriticNetwork(self.input_dims, self.n_actions, self.f1_critic, 
                              self.f2_critic, self.name+"_"+ "critic_target", 
                              self.chkpt_dir, lr= self.lr_critic)
    
    #hard copy of weights from online to target networks
    self.update_network_parameters(tau=1)  

    
    self.memory_buffer = ReplayBuffer( self.max_mem, self.input_dims, self.n_actions)

    #instantiate with zeros mean vector b/c it evolves over tiem 
    self.noise = OUActionNoise(mu=np.zeros(self.n_actions))


  def choose_action(self, state):
    #we don't want to calc layer norm statistics for the choose action func (since we are using batch/layer norm)
    #and we only want to do that during our learning
    self.actor.eval()


    out = self.actor.forward(state).detach().numpy() + self.noise()
    
    self.actor.train()
    
    return out  #what the fuck ? why do I return an array of 4 items since it is not an action??
                    #ANSWER: It actually is an action, we were working on the continuous environment... ....


  def store_transition(self, state, action, reward, next_state, done):
    self.memory_buffer.store_transition(state, action, reward, next_state, done)

  def save_models(self):
    self.actor.save_checkpoint()
    self.actor_target.save_checkpoint()

    self.critic.save_checkpoint()
    self.critic_target.save_checkpoint()

  def load_models(self):
    self.actor.load_checkpoint()
    self.actor_target.save_checkpoint()

    self.critic.load_checkpoint()
    self.critic_target.save_checkpoint()


  def sample_memory(self):
    state, action, reward, next_state, done = self.memory.sample_buffer(self.batch_size)
    states = T.tensor(state).to(self.actor.device)
    actions = T.tensor(action).to(self.actor.device)
    rewards = T.tensor(reward).to(self.actor.device)
    next_states = T.tensor(next_state).to(self.actor.device)
    dones = T.tensor(done).to(self.actor.device)
    return states, actions, rewards, next_states, dones

  
  #self.update_network_parameters(tau=1)  
  def update_network_parameters(self, tau):
    '''
    No other arguments are necessary because they will be taken care of inside the class
    by referencing self.model. However I will use a model argument here 
    '''  
    state_dict_actor = self.actor.state_dict()
    state_dict_actor_target = self.actor_target.state_dict()

    state_dict_critic = self.critic.state_dict()
    state_dict_critic_target = self.critic_target.state_dict()


    for name, param in state_dict_actor.items():
      transformed_param =  ( param * tau )  +  ( (1-tau) * state_dict_actor_target[name] ) 
      state_dict_actor_target[name].copy_(transformed_param)


    for name, param in state_dict_critic.items():
      transformed_param =  ( param * tau )  +  ( (1-tau) * state_dict_critic_target[name] ) 
      state_dict_critic_target[name].copy_(transformed_param)


    #Generally If I would like to make it work with batch norm:
    #self.actor_target.load_state_dict(critic_state_dict, strict=False
    #but it is ntot really recommended because Named parameters do not include running mean and running average

  def learn(self):
    #Q: Critic, mu: Actor => WORKS 

    if self.memory_buffer.mem_cntr < self.batch_size:
        return

    self.actor.optimizer.zero_grad()
    self.critic.optimizer.zero_grad()

    states, actions, rewards, next_states, dones = self.memory_buffer.sample_buffer(self.batch_size)


    #dones = torch.reshape(torch.tensor(dones), (1, 2, 1).to(self.actor.device))
    dones = torch.reshape(torch.tensor(dones).to(self.actor.device), (1, self.batch_size, 1))


    yi = torch.tensor(rewards).unsqueeze(1)  + self.gamma * self.critic_target.forward(next_states, self.actor_target.forward(next_states))


    q = self.critic.forward(states, actions) 

    #adding the terminal states's value == 0 (keep forgetting about this SHIT)
    q[dones] = 0.0

    critic_loss = (1 / self.batch_size ) * (torch.sum((yi - q ) ** 2) )

    critic_loss.backward()

    self.critic.optimizer.step()

    actor_loss =  torch.sum(-self.critic.forward( states, self.actor.forward(states)))  / self.batch_size  

    actor_loss.backward()

    self.actor.optimizer.step()


    #needs soft param update
    self.update_network_parameters(tau=self.tau)




In [11]:
n_eps = 50
agent = DDPGAgent(env.reset().shape, 2)
scores = []
for e in range(n_eps):
  done, state, score = False, env.reset(), 0
  agent.noise.reset()
  while not done:
    action = agent.choose_action(state)  #Do not understand how we will actually select discrete actions EDIT: it's actually 
    next_state, reward, done, _ = env.step(action.squeeze(0))    #the continuous envronmenet! I had misunderstood that 
    agent.store_transition(state, action, reward, next_state, done)
    agent.learn()
    score += reward
  scores.append(score)
  avg_score = np.mean(scores[-100: ])
  print(f"Episode {e} , Average 100 episode score: {avg_score}")

Episode 0 , Average 100 episode score: -263.51780684547657
Episode 1 , Average 100 episode score: -222.7616051484966
Episode 2 , Average 100 episode score: -402.22043294000355
Episode 3 , Average 100 episode score: -414.76470857168135
Episode 4 , Average 100 episode score: -453.3988424247817
Episode 5 , Average 100 episode score: -426.3000763208524
Episode 6 , Average 100 episode score: -392.13601287653256
Episode 7 , Average 100 episode score: -408.994269055839
Episode 8 , Average 100 episode score: -378.04496494468543
Episode 9 , Average 100 episode score: -381.1659938734818
Episode 10 , Average 100 episode score: -404.3421416720755
Episode 11 , Average 100 episode score: -380.97757868492187
Episode 12 , Average 100 episode score: -394.7040718787882
Episode 13 , Average 100 episode score: -400.95742376171177
Episode 14 , Average 100 episode score: -383.8160692771971
Episode 15 , Average 100 episode score: -378.1493974056363
Episode 16 , Average 100 episode score: -361.9074599954917
E